In [3]:
#This code allows you to evaluate which players are the most influential for a team during the calendar year 
#using data from Understat.com we will answer the following questions:
# 1) Who are the 5 most influential players for Goal Events? (Goals + Assists)
# 2) How is the performance of the 5 most influential players during the season (Month by month and by minutes played)
# 3) From the player that scores the most: Where does he prefers to shoot and also where does he score the most goals?
# 4) Who is the best "duo" (Goals and assisted each other the most)
#Importing essential packages at this point

import requests
from bs4 import BeautifulSoup
import json

import pandas as pd
import numpy as np
from datetime import datetime

In [4]:
#Webscrapping understat.com. 

#The section of code for webscrapping was created following McKay Johns youtube videos.

#scrape a single game shots. 
base_url = 'https://understat.com/team/'
match = str(input('Please enter the Team: '))
url = base_url+match+'/'+str(datetime.now().year-1)
print(url) #To make sure thing are going good

Please enter the Team: bayer_leverkusen
https://understat.com/team/bayer_leverkusen/2023


In [5]:
#Use requests to get the webpage and BeautifulSoup to parse the page
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')
scripts = soup.find_all('script')

In [6]:
#get only the shotsData
strings = scripts[1].string

In [7]:
# strip unnecessary symbols and get only JSON data 
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
data = json.loads(json_data)

In [8]:
id1 = []
Played = []
for index in range(len(data)):
    for key in data[index]:
        if key == 'id':
            id1.append(data[index][key])
        if key == 'isResult':
            Played.append(data[index][key])

In [9]:
col_names = ['MatchID','Played']
MatchSetting = pd.DataFrame([id1,Played],index=col_names)
MatchSetting = MatchSetting.T
MatchSetting=MatchSetting[MatchSetting['Played']==True]
match_base_url = 'https://understat.com/match/'

MatchWeb=match_base_url+MatchSetting['MatchID']
MatchSetting['MatchWeb']=MatchWeb


#Create a list for each variable we want to take out of understat.com
x = []
y = []
xG = []
result = []
team = []
HA = []
id1 = []
player = []
player_assisted = []
shot_type = []
situation = []
minute = []
datetime = []

In [10]:
MatchSetting=MatchSetting.reset_index()
MatchSetting=MatchSetting.drop('index', axis=1)
print(MatchSetting.head())

  MatchID Played                           MatchWeb
0   23066   True  https://understat.com/match/23066
1   23080   True  https://understat.com/match/23080
2   23084   True  https://understat.com/match/23084
3   23092   True  https://understat.com/match/23092
4   23108   True  https://understat.com/match/23108


In [11]:
#Use requests to get the webpage and BeautifulSoup to parse the page

for index in range(len(MatchSetting)):
    res = requests.get(MatchSetting['MatchWeb'][index])
    soup = BeautifulSoup(res.content, 'lxml')
    scripts = soup.find_all('script')
    #get only the shotsData
    strings = scripts[1].string
    # strip unnecessary symbols and get only JSON data 
    ind_start = strings.index("('")+2 
    ind_end = strings.index("')") 
    json_data = strings[ind_start:ind_end] 
    json_data = json_data.encode('utf8').decode('unicode_escape')

    #convert string to json format
    data = json.loads(json_data)
    data_away = data['a']
    data_home = data['h']

    for index in range(len(data_home)):
        for key in data_home[index]:
            if key == 'date':
                datetime.append(data_home[index][key])
            if key == 'minute':
                minute.append(data_home[index][key])
            if key == 'match_id':
                id1.append(data_home[index][key])
            if key == 'X':
                x.append(data_home[index][key])
            if key == 'Y':
                y.append(data_home[index][key])
            if key == 'h_team':
                team.append(data_home[index][key])
            if key == 'xG':
                xG.append(data_home[index][key])
            if key == 'result':
                result.append(data_home[index][key])
            if key == 'player':
                player.append(data_home[index][key])
            if key == 'player_assisted':
                player_assisted.append(data_home[index][key])
            if key == 'h_a':
                HA.append(data_home[index][key])
            if key == 'shotType':
                shot_type.append(data_home[index][key])            
                    
    for index in range(len(data_away)):
        for key in data_away[index]:
            if key == 'date':
                datetime.append(data_away[index][key])
            if key == 'minute':
                minute.append(data_away[index][key])
            if key == 'match_id':
                id1.append(data_away[index][key])
            if key == 'X':
                x.append(data_away[index][key])
            if key == 'Y':
                y.append(data_away[index][key])
            if key == 'a_team':
                team.append(data_away[index][key])
            if key == 'xG':
                xG.append(data_away[index][key])
            if key == 'result':
                result.append(data_away[index][key])
            if key == 'player':
                player.append(data_away[index][key])
            if key == 'player_assisted':
                player_assisted.append(data_away[index][key])
            if key == 'h_a':
                HA.append(data_away[index][key])
            if key == 'shotType':
                shot_type.append(data_away[index][key])
                 

In [12]:
col_names = ['date','minute','MatchID','x','y','xG','result','team','Home_Away','player','player_assisted','Shot type']
df = pd.DataFrame([datetime,minute,id1,x,y,xG,result,team,HA,player,player_assisted,shot_type],index=col_names)
df = df.T

In [13]:
#Saving Dataframe so I don't have to webscape the data in the future
df.to_csv('C:/Users/verde/Desktop/data/'+match+'.csv', index=False) 

#Re-opening it
df = pd.read_csv('C:/Users/verde/Desktop/data/'+match+'.csv')
df.head()

,date,minute,MatchID,x,y,xG,result,team,Home_Away,player,player_assisted,Shot type
0,2023-08-19 13:30:00,1,23066,0.723,0.570,0.015755,SavedShot,Bayer Leverkusen,h,Granit Xhaka,NaN,LeftFoot
1,2023-08-19 13:30:00,7,23066,0.762,0.445,0.022466,MissedShots,Bayer Leverkusen,h,Jonas Hofmann,NaN,LeftFoot
2,2023-08-19 13:30:00,23,23066,0.939,0.451,0.564749,Goal,Bayer Leverkusen,h,Jeremie Frimpong,Victor Boniface,RightFoot
3,2023-08-19 13:30:00,33,23066,0.860,0.594,0.086157,BlockedShot,Bayer Leverkusen,h,Álex Grimaldo,NaN,RightFoot
4,2023-08-19 13:30:00,34,23066,0.925,0.476,0.339712,Goal,Bayer Leverkusen,h,Jonathan Tah,Jonas Hofmann,Head


In [14]:
df.isna().value_counts()

date   minute  MatchID  x      y      xG     result  team   Home_Away  player  player_assisted  Shot type
False  False   False    False  False  False  False   False  False      False   False            False        705
                                                                               True             False        208
Name: count, dtype: int64

In [15]:
df['player_assisted'] = df['player_assisted'].fillna('None') #We will assume None for the NaN values on the assist

df.isna().value_counts()

date   minute  MatchID  x      y      xG     result  team   Home_Away  player  player_assisted  Shot type
False  False   False    False  False  False  False   False  False      False   False            False        913
Name: count, dtype: int64

In [16]:
df.dtypes #Checking types.

date                object
minute               int64
MatchID              int64
x                  float64
y                  float64
xG                 float64
result              object
team                object
Home_Away           object
player              object
player_assisted     object
Shot type           object
dtype: object

In [17]:
#Resetting types to make sure I can work with them later
df['xG'] = df['xG'].astype(float)
df['x'] =  df['x'].astype(float)
df['y'] =  df['y'].astype(float)

#Cleaning Data
df['Shot type'] = df['Shot type'].str.replace('LeftFoot', 'Left Foot')
df['Shot type'] = df['Shot type'].str.replace('RightFoot', 'Right Foot')
df['Shot type'] = df['Shot type'].str.replace('Head', 'Header')
df['Shot type'] = df['Shot type'].str.replace('OtherBodyPart', 'Other')

#Adding some values of interest
df['date'] = pd.to_datetime(df['date']).dt.date
df['Month'] = pd.to_datetime(df['date']).dt.month

In [18]:
df.dtypes

date                object
minute               int64
MatchID              int64
x                  float64
y                  float64
xG                 float64
result              object
team                object
Home_Away           object
player              object
player_assisted     object
Shot type           object
Month                int32
dtype: object

In [19]:
#Adding Goals and Assist
def set_goal_and_assist(row):
    if (row['result']=='Goal'):
        row['Goal'] = 1
        
        if (row['player_assisted']!='None'):
            row['Assist'] = 1
        else:
            row['Assist'] = 0
    
    else:
        row['Goal'] = 0
        row['Assist'] = 0

    return row

In [20]:
#I have to adjust the x and y to correctly fit the pitch in Python. 
#Very important: You will see this reflected later...Since we are plotting a VERTICAL field. X and Y will swap with each other
#Feel free to change the name of the column, I personally did not do it.

for index, row in df.iterrows():
    if df.loc[index,'Home_Away']=='h':
        df.loc[index,'x']=row['x']*120+1
        df.loc[index,'y'] = row['y']*70+5
        df.loc[index,'xG'] = row['xG']

    elif df.loc[index,'Home_Away']=='a':
        df.loc[index,'x']=row['x']*121
        df.loc[index,'y'] = row['y']*78
        df.loc[index,'xG'] = row['xG']

#This helps to ensure that I can get the info from the team I originally introduced at the URL portion
Team=match.replace("_", " ").title()
dfStudy = df[df['team']==Team]  #Study Team
dfOp = df[df['team']!=Team]  #Opposite teams

dfStudy = dfStudy.apply(set_goal_and_assist, axis=1)